In [2]:
import datetime, time, math, os, json, shutil, asyncio, networkx as nx
from utils.gitutils import create_pull_request, clone_repo, create_branch, wipe_repo, prepare_repo
from utils.agent import Agent, GenerationConfig, Interaction, Team
from utils.stringutils import arr_from_sep_string, extract_markdown_blocks, markdown_to_dict, find_most_similar_substring
from utils.filetreeutils import FileTree, write_file_tree
from utils.listutils import flatten
from utils.frameworkutils import DartAnalyzer
from dotenv import load_dotenv
from utils.graphutils import loose_level_order, collapsed_level_order
from utils.frameworkutils import Framework

In [3]:
load_dotenv()  # Load environment variables from .env

GITHUB_TOKEN = os.getenv('GITHUB_TOKEN')
API_KEY = os.getenv('API_KEY')

In [4]:
repo_url = 'https://github.com/6165-MSET-CuttleFish/TeamTrack'
source = Framework.FLUTTER
feature = f"""## Name
Organizations
## Description
Organizations are a way to group users together. They can be used to automatically share events with the members of the organization whenever they are uploaded to the cloud. \
Organizations consist of a name, team number, and a backend-generated unique id. They can also have a profile picture, description, and location. \
Members can also opt to be publicly listed as a member of the organization or not. \
"""

In [23]:
# Clone the repo and make a new branch
repo = clone_repo(repo_url)
base_branch = repo.active_branch.name
created_branch = f"modification-{base_branch}"
local_repo_path = str(repo.working_dir)
working_dir_path = f'{local_repo_path}\\{source.get_working_dir()}'

In [6]:
source_swe_summarizer = Agent(
    model_name="gemini-1.5-flash-001",
    api_key=API_KEY,
    name="source_swe_analyzer",
    generation_config=GenerationConfig(temperature=0.7),
    system_prompt=f'''You are a {source} software engineer. Your job is to summarize the functionality of provided code files. Structure your response as follows:

# filename.ext:

$7 sentence summary of the functionality/contents of the file$
'''
)

pm = Agent(
    model_name="gemini-1.5-pro-001",
    api_key=API_KEY,
    name="pm",
    generation_config=GenerationConfig(temperature=0.3),
    system_prompt=f"""You are a high-level technical project manager tasked with the project of adding functionality to a {source} repository."""
)

source_swe_coder = Agent(
    model_name="gemini-1.5-flash-001",
    api_key=API_KEY,
    name="source_swe_coder",
    generation_config=GenerationConfig(temperature=0.5),
    system_prompt=f"You are a {source} software engineer. Your job is to write code to implement a specific feature. Respond with code and nothing else. No explanations needed."
)

team = Team(source_swe_summarizer, pm, source_swe_coder)

In [7]:
analyzer = source.get_analyzer(working_dir_path)

source_dependency_graph = analyzer.buildDependencyGraph()

In [8]:
# loose level order
eval_order = loose_level_order(source_dependency_graph)[::-1]
for level in eval_order:
    print(level)
print(f"Number of levels: {len(eval_order)}")

['models\\StatConfig.dart', 'models\\AppModel.dart']
['models\\GameModel.dart']
['models\\ScoreModel.dart']
['components\\misc\\PlatformGraphics.dart', 'functions\\Extensions.dart']
['components\\users\\PFP.dart', 'components\\statistics\\BarGraph.dart']
['components\\users\\UsersRow.dart', 'providers\\Auth.dart', 'components\\scores\\Incrementor.dart', 'functions\\Statistics.dart', 'components\\scores\\ScoreSummary.dart']
['views\\home\\match\\MatchView.dart']
['functions\\Functions.dart', 'components\\statistics\\PercentChange.dart']
['components\\scores\\ScoreTimeline.dart', 'components\\scores\\ScoreRangeSummary.dart', 'models\\Change.dart']
['components\\scores\\ScoringElementStats.dart', 'components\\statistics\\CheckList.dart', 'components\\misc\\CardView.dart', 'views\\home\\match\\ExampleMatchRow.dart', 'views\\home\\match\\MatchConfig.dart', 'views\\home\\change\\ChangeConfig.dart', 'views\\home\\match\\MatchRow.dart', 'components\\misc\\EmptyList.dart', 'views\\home\\change\

In [9]:
(source_file_tree := FileTree.from_dir(working_dir_path))

├── api\
│   ├── README.MD
├── components\
│   ├── misc\
│   │   ├── CardView.dart
│   │   ├── Collapsible.dart
│   │   ├── EmptyList.dart
│   │   ├── InfoPills.dart
│   │   ├── PlatformGraphics.dart
│   ├── scores\
│   │   ├── Incrementor.dart
│   │   ├── ScoreCard.dart
│   │   ├── ScoreRangeSummary.dart
│   │   ├── ScoreSummary.dart
│   │   ├── ScoreTimeline.dart
│   │   ├── ScoringElementStats.dart
│   ├── statistics\
│   │   ├── BarGraph.dart
│   │   ├── CheckList.dart
│   │   ├── PercentChange.dart
│   ├── users\
│   │   ├── PFP.dart
│   │   ├── UsersRow.dart
│   ├── AutonomousDrawingTool.dart
├── functions\
│   ├── APIMethods.dart
│   ├── Extensions.dart
│   ├── Functions.dart
│   ├── GPTModel.dart
│   ├── ResponseModel.dart
│   ├── Statistics.dart
├── models\
│   ├── AppModel.dart
│   ├── Change.dart
│   ├── GameModel.dart
│   ├── GPTModel.dart
│   ├── ScoreModel.dart
│   ├── StatConfig.dart
├── providers\
│   ├── Auth.dart
│   ├── PushNotifications.dart
│   ├── Theme.dart
│   ├

In [10]:
# Summarize the original repo
from tqdm import tqdm

async def summarize_group(group):
    async def summarize(node):
        name = source_file_tree.nodes[node]['name']
        content = source_file_tree.nodes[node]['content']
        message = f"{name}\n```\n{content}\n```"
        return await team.async_chat_with_agent(
            agent_name='source_swe_analyzer',
            message=message,
            context_keys=[f"summary_{neighbor}" for neighbor in source_dependency_graph[node]],
            save_keys=[f"summary_{node}", "all"],
            prompt_title=f"Summary of {node}"
            )
    tasks = [summarize(node) for node in group]
    responses = await asyncio.gather(*tasks)
    for i, response in enumerate(responses):
        source_file_tree.nodes[group[i]]["summary"] = response

for level in tqdm(eval_order):
    await summarize_group(level)
    time.sleep(0.5)

100%|██████████| 19/19 [01:19<00:00,  4.17s/it]


In [11]:
prompt = f"""This is the file tree for the original {source} repo:
```
{source.get_working_dir()}\\
{source_file_tree}
```
Given the prior context, summarize the functionality of the entire repository succinctly.
"""
print(team.chat_with_agent('source_swe_analyzer', prompt, context_keys=["all"], save_keys=["all"], prompt_title=f"""Repository Summary.
File Tree provided:
```
{source.get_working_dir()}\\
{source_file_tree}
```
"""))

This repository contains the source code for a Flutter application designed to help manage and analyze data for FIRST Robotics Competition events. The app allows users to create and manage events, track team scores, view match results, share events with other users, and analyze team performance. It leverages Firebase for data storage and authentication, providing features for user management, event sharing, and data synchronization. The app also utilizes Google ML Kit for image analysis, enabling users to import match schedules from images. It incorporates AI functionality through OpenAI's ChatGPT API to provide team recommendations and insights.  The app also includes features for managing user presence, push notifications, and theme customization. 



In [12]:
curr_string = '\n'.join([message['parts'][0] for interaction in team.context_threads['all'] for message in interaction.to_dict()])
(curr_tokens := pm.model.count_tokens(curr_string).total_tokens)

19010

In [13]:
would_be_string = '\n'.join([source_file_tree.nodes[node]['content'] for node in source_dependency_graph.nodes])
(would_be_tokens := pm.model.count_tokens(would_be_string).total_tokens)

122951

In [14]:
(percent_tokens_reduction := 1 - (curr_tokens / would_be_tokens))

0.8453855601011785

In [15]:
response_format = f"""
You may choose ACTIONS from the following list:
- | WRITE | $FILE_PATH$ | $PROMPT$ (PROMPT should inform how the new FILE_PATH should be written)
- | EDIT | $FILE_PATH$ | $PROMPT$ (Edit the provided file to fulfill the prompt)
- | DELETE | $FILE_PATH$ (Delete the provided file)
----------
Here is a sample response:
```
| WRITE | {source.get_working_dir()}\\path\\to\\newfile.ext | Implement ... with the following properties:
    - property1
    - property2
    - dependency1
    - dependency2
| EDIT | {source.get_working_dir()}\\path\\to\\file.ext | Fix the bug ... by doing:
    - step1
    - step2
| DELETE | {source.get_working_dir()}\\path\\to\\some\\file.ext | Reasons for deletion:
    - reason1
    - reason2
```
"""

# Create new file tree
prompt = f'''
You are to build the following feature:
{feature}
------------------
{response_format}
------------------
You need to follow the response template structure exactly (MY JOB DEPENDS ON IT)
'''
print(response := team.chat_with_agent("pm", prompt, context_keys=["all"], save_keys=["all"], prompt_title="Feature Creation Steps"))

```
| WRITE | lib/models/Organization.dart | Implement the `Organization` class with the following properties:
    - `id`: String (backend-generated unique ID)
    - `name`: String
    - `teamNumber`: int
    - `profilePicture`: String (URL, optional)
    - `description`: String (optional)
    - `location`: String (optional)
| WRITE | lib/providers/OrganizationProvider.dart | Implement the `OrganizationProvider` class. This class should:
    - Use `ChangeNotifier` to notify listeners of changes.
    - Handle creating, updating, and deleting organizations in Firebase.
    - Handle adding and removing organization members in Firebase.
    - Provide methods for retrieving organization data from Firebase.
| WRITE | lib/views/organization/OrganizationPage.dart | Implement the `OrganizationPage` widget. This widget should:
    - Display a list of organizations the user is a member of.
    - Allow users to create a new organization.
    - Navigate to the `OrganizationView` when an organizatio

In [16]:
md = extract_markdown_blocks(response)[0]
blocks = md.split("|")
actions = []
i = 0
while i < len(blocks):
    block = blocks[i].strip()
    if block != "WRITE" and block != "EDIT" and block != "DELETE":
        i += 1
        continue
    action, file, prompt = blocks[i:i + 3]
    action = action.strip()
    file = file.strip()[len(source.get_working_dir()) + 1:]
    prompt = prompt.strip()
    actions.append((action, file, prompt))
    i += 3
for action in actions:
    print(action)

('WRITE', 'models/Organization.dart', 'Implement the `Organization` class with the following properties:\n    - `id`: String (backend-generated unique ID)\n    - `name`: String\n    - `teamNumber`: int\n    - `profilePicture`: String (URL, optional)\n    - `description`: String (optional)\n    - `location`: String (optional)')
('WRITE', 'providers/OrganizationProvider.dart', 'Implement the `OrganizationProvider` class. This class should:\n    - Use `ChangeNotifier` to notify listeners of changes.\n    - Handle creating, updating, and deleting organizations in Firebase.\n    - Handle adding and removing organization members in Firebase.\n    - Provide methods for retrieving organization data from Firebase.')
('WRITE', 'views/organization/OrganizationPage.dart', 'Implement the `OrganizationPage` widget. This widget should:\n    - Display a list of organizations the user is a member of.\n    - Allow users to create a new organization.\n    - Navigate to the `OrganizationView` when an orga

In [17]:
source_swe_coder.model.count_tokens('\n'.join(message['parts'][0] for interaction in team.context_threads['all'] for message in interaction.to_dict())).total_tokens

19442

In [18]:
# Cache the context threads
factor = 32768 / source_swe_coder.model.count_tokens('\n'.join(message['parts'][0] for interaction in team.context_threads['all'] for message in interaction.to_dict())).total_tokens
# eg. if factor is 3.2, round to 4
factor = math.ceil(factor)
thread = team.context_threads['all'].copy()
thread *= factor
source_swe_coder.model.count_tokens('\n'.join(message['parts'][0] for interaction in thread for message in interaction.to_dict())).total_tokens

38853

In [24]:
def write(file_path, prompt):
    response = source_swe_coder.chat(f"Instructions for {file_path}:\n{prompt}")
    md = extract_markdown_blocks(response)[0]
    path = f"{working_dir_path}\\{file_path}"
    if not os.path.exists(os.path.dirname(path)):
        os.makedirs(os.path.dirname(path))
    with open(path, 'w') as f:
        f.write(md)

def edit(file_path, prompt):
    with open(f"{working_dir_path}\\{file_path}", 'r') as f:
        content = f.read()
    response = source_swe_coder.chat(f"""{prompt}
We will solve this problem by diving edits into small sub-edits. Structure your response as follows:
# Step 1
$Brief description of the first sub-change$
## Short Old Code Snippet 1
```
$short old code snippet that needs to changed$
```
## New Code Snippet 1
```
$the new code snippet needed to fulfill the prompt$
```
------------------
...
...
------------------
# Step N
$Brief description of the Nth sub-change$
## Short Old Code Snippet N
```
$...$
```
## New Code Snippet N
```
$...$
```.  
We have a very advanced algorithm to find the closest match to the old code snippet, so you only need to provide a few lines of padding context around the old code snippet.
Respond with as many or as few snippets as you need but MOST IMPORTANTLY: MAKE SURE THAT EACH SNIPPET IS SHORT (MY JOB DEPENDS ON IT).
Additionally, DO NOT RESPOND WITH A DIFF FILE OR IN A DIFFERENT FORMAT, MY JOB ALSO DEPENDS ON IT""", custom_context=[Interaction(f"Contents of {file_path}", content)])
    blocks = extract_markdown_blocks(response)
    # even blocks are old code, odd blocks are new code
    old_codes = [block for i, block in enumerate(blocks) if i % 2 == 0]
    new_codes = [block for i, block in enumerate(blocks) if i % 2 == 1]
    closest_matches = []
    for i in range(len(old_codes)):
        old_code = old_codes[i]
        closest_match, distance = find_most_similar_substring(content, old_code)
        print(f"Old Code: {old_code}")
        print(f"Closest match: {closest_match}")
        closest_matches.append(closest_match)
    for i in range(len(old_codes)):
        closest_match = closest_matches[i]
        new_code = new_codes[i]
        content = content.replace(closest_match, new_code)
        # clear the contents of the file
        with open(f'{working_dir_path}\\{file_path}', 'w') as f:
            f.seek(0)
            f.truncate()
            f.write(content)

def delete(file_path):
    os.remove(f"{working_dir_path}\\{file_path}")

In [25]:
from utils.errorutils import repeat_until_finish
from tqdm import tqdm

for i in tqdm(range(len(actions))):
    action, file, prompt = actions[i]
    if action == 'WRITE':
        repeat_until_finish(lambda: write(file, prompt), max_retries=2)
    elif action == 'EDIT':
        repeat_until_finish(lambda: edit(file, prompt), max_retries=2)
    elif action == 'DELETE':
        delete(file)

 57%|█████▋    | 4/7 [00:18<00:15,  5.31s/it]

Old Code: enum Tab {
  events,
  inbox,
  blocked_users,
  templates,
}
Closest match: enum Tab {
  events,
  inbox,
  blocked_users,
  templates,
}
Old Code: Widget title() {
    switch (tab) {
      case Tab.inbox:
        return Text("Inbox");
      case Tab.blocked_users:
        return Text("Blocked Users");
      case Tab.templates:
        return Text("Templates");
      case Tab.events:
        return Center(child:Text("TeamTrack",textScaleFactor: 1.5,));
    }
  }
Closest match:   Widget title() {
    switch (tab) {
      case Tab.inbox:
        return Text("Inbox");
      case Tab.blocked_users:
        return Text("Blocked Users");
      case Tab.templates:
        return Text("Templates");
      case Tab.events:
        return Center(child:Text("TeamTrack",textScaleFactor: 1.5,));
    }
  }
Old Code: Widget body() {
    switch (tab) {
      case Tab.inbox:
        return Inbox();
      case Tab.blocked_users:
        return BlockList();
      case Tab.templates:
        ret

 71%|███████▏  | 5/7 [00:54<00:32, 16.48s/it]

Old Code: Column(
                    mainAxisSize: MainAxisSize.min,
                    children: [
                      ListTile(
                        leading: Icon(Icons.list,
                            color: Theme.of(context).colorScheme.onBackground),
                        title: Text("Events"),
                        onTap: () {
                          setState(() => tab = Tab.events);
                          Navigator.of(context).pop();
                        },
                      ),
                      if (!NewPlatform.isWeb)
                        ListTile(
                          leading: Icon(CupertinoIcons.square_stack,
        color: Theme.of(context).colorScheme.onBackground),
                          title: Text("Templates"),
                          onTap: () {
                            setState(() => tab = Tab.templates);
                            Navigator.of(context).pop();
                          },
                        ),
         

 86%|████████▌ | 6/7 [00:56<00:11, 11.46s/it]

Old Code: List<Event> events = [];
  List<Event> sharedEvents = [];
  String? token;
  List<Event> inbox = [];
  List<TeamTrackUser> blockedUsers = [];
Closest match:   List<Event> events = [];
  List<Event> sharedEvents = [];
  String? token;
  List<Event> inbox = [];
  List<TeamTrackUser> blockedUsers = [];


100%|██████████| 7/7 [01:02<00:00,  8.88s/it]

Old Code: await _firebaseAuth.createUserWithEmailAndPassword(
          email: email, password: password);
Closest match:         await _firebaseAuth.createUserWithEmailAndPassword(
            email: email, password: password);
Old Code: await _firebaseAuth.currentUser?.linkWithCredential(credential);
Closest match:         await _firebaseAuth.currentUser?.linkWithCredential(credential);
Old Code: var result = await _firebaseAuth.signInWithCredential(credential);
      addToken();
      return result;
Closest match:       var result = await _firebaseAuth.signInWithCredential(credential);
      addToken();
      return result;
Old Code: await _firebaseAuth.signInWithEmailAndPassword(
          email: email, password: password);
      addToken();
      return "Signed in";
Closest match:       await _firebaseAuth.signInWithEmailAndPassword(
          email: email, password: password);
      addToken();
      return "Signed in";
